In [200]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
import pickle
import sklearn
import statsmodels.tsa.statespace as sm

In [3]:
df = pd.read_csv('./../data/zri_acs_bikeshare_merged.csv')

In [5]:
df['zip'] = df['zip'].map(lambda x: str(x))

In [6]:
df['zip'] = df['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [10]:
df['datetime']=df['datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [15]:
df = df.set_index('datetime')

In [16]:
df

,zip,zri,year,month,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,...,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
datetime,,,,,,,,,,,,,,,,,,,,,
2014-01-01,01013,928.0,2014,1,0.728315,0.020951,0.013732,0.219830,0.000000,0.004959,...,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
2014-02-01,01013,931.0,2014,2,0.727052,0.021179,0.013708,0.220867,0.000000,0.004917,...,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
2014-03-01,01013,934.0,2014,3,0.725788,0.021408,0.013685,0.221904,0.000000,0.004874,...,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
2014-04-01,01013,929.0,2014,4,0.724525,0.021636,0.013661,0.222940,0.000000,0.004832,...,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
2014-05-01,01013,929.0,2014,5,0.723261,0.021864,0.013638,0.223977,0.000000,0.004789,...,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,99654,1277.0,2019,8,0.796036,0.008872,0.014941,0.049974,0.049727,0.003070,...,60057.000000,0.102326,0.049199,33.566667,0.475244,0.399400,0.018076,0.0,0.0,0
2019-09-01,99654,1271.0,2019,9,0.795709,0.008765,0.014871,0.050125,0.049560,0.003065,...,60203.000000,0.102254,0.049419,33.575000,0.475511,0.399650,0.018306,0.0,0.0,0
2019-10-01,99654,1299.0,2019,10,0.795381,0.008658,0.014801,0.050275,0.049393,0.003060,...,60349.000000,0.102181,0.049640,33.583333,0.475778,0.399900,0.018537,0.0,0.0,0


In [238]:
y

,zip,zri
datetime,,
2014-01-01,01013,928.0
2014-02-01,01013,931.0
2014-03-01,01013,934.0
2014-04-01,01013,929.0
2014-05-01,01013,929.0
...,...,...
2019-08-01,99654,1277.0
2019-09-01,99654,1271.0
2019-10-01,99654,1299.0


In [239]:
x

,zip,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,...,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
datetime,,,,,,,,,,,,,,,,,,,,,
2014-01-01,01013,0.728315,0.020951,0.013732,0.219830,0.000000,0.004959,0.230010,0.311464,0.309663,...,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
2014-02-01,01013,0.727052,0.021179,0.013708,0.220867,0.000000,0.004917,0.229526,0.311552,0.310406,...,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
2014-03-01,01013,0.725788,0.021408,0.013685,0.221904,0.000000,0.004874,0.229041,0.311641,0.311150,...,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
2014-04-01,01013,0.724525,0.021636,0.013661,0.222940,0.000000,0.004832,0.228557,0.311729,0.311894,...,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
2014-05-01,01013,0.723261,0.021864,0.013638,0.223977,0.000000,0.004789,0.228072,0.311818,0.312638,...,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,99654,0.796036,0.008872,0.014941,0.049974,0.049727,0.003070,0.282075,0.308002,0.320101,...,60057.000000,0.102326,0.049199,33.566667,0.475244,0.399400,0.018076,0.0,0.0,0
2019-09-01,99654,0.795709,0.008765,0.014871,0.050125,0.049560,0.003065,0.282024,0.308218,0.319399,...,60203.000000,0.102254,0.049419,33.575000,0.475511,0.399650,0.018306,0.0,0.0,0
2019-10-01,99654,0.795381,0.008658,0.014801,0.050275,0.049393,0.003060,0.281973,0.308434,0.318697,...,60349.000000,0.102181,0.049640,33.583333,0.475778,0.399900,0.018537,0.0,0.0,0


In [133]:
zips = list(df['zip'].unique())

y = df.drop(columns = ['year', 'month']).loc[:,['zip', 'zri']]
x = df.drop(columns = ['year', 'month', 'zri'])

In [205]:
pred_dict = {}
conf_int_dict = {}
coef_dict = {}

In [206]:
start = datetime.now()
for item in zips:
    curr_y = y[y['zip']==item]
    curr_x = x[x['zip']==item].drop(columns = 'zip')
    curr_x_train = curr_x.iloc[0:60,:]
    curr_x_test = curr_x.iloc[60:,:]
    
    
    curr_model = sm.sarimax.SARIMAX(curr_y['zri'], exog = curr_x, order = (1,1,0), seasonal_order=(0, 0, 0, 0)).fit()
    
    pred_dict[item] = curr_model.predict(start = 1, end = 71, exog = curr_x_test, dynamic = 59)
    coef_dict[item] = curr_model.params

elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_mo

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:5

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

...

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_mo

0:11:00.784188


In [207]:
fig = go.Figure()
fig.add_trace(go.Line(x = curr_train.index, y = curr_train['zri'], mode = 'lines'))
fig.add_trace(go.Line(x = curr_test.index, y = curr_test['zri'], mode = 'lines'))
fig.add_trace(go.Line(x = pred.index, y = pred, mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,0], mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,1], mode = 'lines', fill = 'tonexty'))

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [210]:
pred_df = pd.DataFrame(pred_dict)

In [222]:
actual_dict = {}

In [223]:
for item in zips:
    curr_y = y[y['zip']==item]
    actual_dict[item] = curr_y.iloc[60:,1]

In [225]:
actual_df = pd.DataFrame(actual_dict)

In [230]:
resid_df = (pred_df.iloc[59:,:] - actual_df).T

In [231]:
resid_df

,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01
01013,11.634478,15.340554,10.548157,5.990484,7.542664,-0.853742,-7.226085,-13.587167,-16.942978,-33.296322,-15.648512,-6.000161
01020,18.470614,19.757559,12.986654,13.197529,17.402665,8.605993,-2.191248,-8.988669,-12.786146,-32.583640,-23.381141,-14.178643
01040,-4.449479,-18.022248,-21.727519,-19.109133,-14.369993,-21.585802,-26.784802,-33.977531,-37.167920,-51.357436,-22.546627,-8.735696
01085,-8.441164,-10.301750,-12.412561,-6.672654,-9.021810,-11.424099,-9.858087,-4.310987,0.224831,13.753917,18.778988,23.801663
01104,1.403088,4.630781,2.172775,4.897665,10.728985,12.622237,13.551529,16.501792,14.464260,-2.566171,-5.592470,-3.616364
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,6.209882,2.135935,-5.114658,-16.475189,-20.904143,-19.375681,-14.873721,-5.388255,8.086944,33.555755,21.020590,8.482950
99501,24.914880,33.901249,42.267159,47.218460,42.892708,37.381821,31.747220,24.029951,12.257441,-23.551984,-30.386075,-37.236650
99504,16.942551,16.447767,9.689686,2.773085,-5.238952,-9.308447,-11.412534,-10.537448,-15.674901,-32.819902,-52.969449,-35.121731
99508,16.981095,15.118475,3.658043,-7.225968,-22.410105,-36.806895,-48.354360,-62.008586,-73.738457,-90.521926,-98.343373,-104.191727


In [240]:
pd.DataFrame(coef_dict)

,01013,01020,01040,01085,01104,01108,01109,01420,01440,01453,...,98409,98444,98466,98498,98499,99207,99501,99504,99508,99654
percent_white,-289.056762,-821.753873,183.256841,79.919851,32.795821,89.037112,-132.415501,-554.736921,-145.692167,-514.841243,...,16.329395,-353.050093,-1138.117055,-858.996477,-257.149314,26.991623,-249.800900,-32.621048,131.532179,-377.857822
percent_black,-26.542016,-173.651703,-25.786706,-54.803286,201.963090,-283.560303,-450.507540,39.140059,-65.449881,-446.643436,...,-62.546441,-66.049886,35.671119,572.324093,399.126871,92.418238,-53.728836,4.628073,321.178847,443.152608
percent_asian,-48.012742,49.177910,43.107284,-45.682323,-240.544000,-14.033896,24.825258,186.807621,-385.432441,-121.695030,...,-2.197410,-130.894497,342.489523,-643.474541,174.248414,-31.509111,150.391337,2.359462,-714.978095,82.741241
percent_hispanic,331.419822,602.700460,-186.274869,13.412006,-80.221572,121.201865,489.669008,286.225179,575.533666,1243.311764,...,35.801399,260.468334,-227.722146,-542.279798,257.300724,-21.811591,113.002082,20.538776,-4.507671,57.936322
percent_native_am,8.491483,-18.375442,-10.276941,15.426750,3.280770,-10.795845,-3.875289,21.382100,8.931323,-65.406269,...,56.358714,6.726993,-77.918649,236.780493,-153.884165,-17.581249,47.572674,3.144584,-521.539425,-326.179486
percent_other_race,27.426024,-49.267442,-9.989013,11.124868,11.710771,30.401175,-0.444390,-17.187005,-3.960449,-45.705320,...,2.029712,44.077766,-52.785039,-30.359728,22.953218,0.676080,2.819160,-1.671871,-30.588481,0.904572
percent_0_17,-5.018629,-340.467645,-141.140736,-34.278930,230.331024,112.352209,-123.881628,147.271340,-13.645797,48.266099,...,31.515794,-26.014277,82.142565,-351.492920,299.806786,-89.219630,-64.057374,-42.458132,-424.847883,-172.841940
percent_18_39,-38.198765,717.096727,120.624072,82.860565,-162.123432,80.771771,131.853150,-23.055164,-167.051918,-47.530330,...,-14.037203,-48.908653,-211.987249,601.761642,-256.871047,20.649913,84.491601,0.015311,-163.243015,361.231075
percent_40_64,-3.362219,52.091370,-24.027742,-120.160403,-263.355966,-136.369805,-32.778428,-74.265239,386.433606,-41.400914,...,10.316693,45.258429,-258.888969,49.379329,165.916955,93.110378,-20.574590,88.147300,334.381467,-295.144643
percent_65+,46.579146,-428.720347,44.544407,71.581489,195.148131,-56.754176,24.806391,-49.964908,-205.731879,40.676333,...,-27.782244,29.667085,388.739130,-299.648049,-208.851021,-24.540661,0.142155,-45.704474,253.735817,106.755510


In [232]:
resid_df.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [234]:
np.sqrt(((resid_df**2).sum().sum())/(1301*12))

49.88677180832918

In [235]:
coef_df = pd.DataFrame(coef_dict)

In [237]:
coef_df.sum(axis=1)/1301

percent_white                       -50.040195
percent_black                        -1.340148
percent_asian                        13.105633
percent_hispanic                     19.373974
percent_native_am                    -3.676238
percent_other_race                    5.326383
percent_0_17                        -35.658472
percent_18_39                         5.665117
percent_40_64                         1.537206
percent_65+                          28.467330
percent_rental_units_vacant          -9.409294
percent_rental_units_occupied         9.420449
percent_graduate_deg                 32.392005
percent_bachelors                    -2.858837
percent_associates                   19.268817
percent_highschool                  -29.418139
percent_less_highschool             -26.796883
percent_commute_public_transport     13.161779
percent_commute_less_30              23.140409
percent_commute_30_to_59            -24.323753
percent_commute_60_to_89             -0.196668
percent_commu